<a href="https://colab.research.google.com/github/prakrutpatel/Context-RCNN-Tortoises/blob/main/Faster_RCNN_Training_on_Serengeti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Make sure tensorflow gpu version 1.15 is installed as there seems to be compatibility issues between tensorflow and numpy with other versions that causes an error while training
!pip install tensorflow_gpu==1.15

## Install other dependencies

In [ ]:
!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

In [ ]:
%load_ext tensorboard

##Restart runtime as this point so we have compatible tensorflow_gpu and numpy versions

#### Install required packages and download Tensorflow Framework from GitHub

In [ ]:
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib pycocotools tf_slim

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

## Optional - If working on Colab, install colab package and mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Prepare `tfrecord` files

This example uses pre-annotated gopher tortoise dataset hosted on my roboflow page.

Because we need one TFRecord file for our training data, and one TFRecord file for our test data, we'll create two separate datasets in Roboflow and generate one set of TFRecords for each.

In [ ]:
!curl -L "https://app.roboflow.com/ds/4DemUlSMP4?key=OPFgu1ApfM" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

**Make sure train and test folders contain '.pbtxt' and '.tfrecord' file**

In [ ]:
# NOTE: Update these TFRecord names to your respective folders and names
test_record_fname = '/content/drive/MyDrive/B004_context_rcnn_memory_test'
train_record_fname = '/content/drive/MyDrive/B004_context_rcnn_memory_train'
label_map_pbtxt_fname = '/content/tortoise_label_map.pbtxt'

## Download base model

We will use Faster R-CNN ResNet 101 trained on serengeti data as our starting point. Transfer Learning will allow us to modify existing weights and configurations to work with our custom dataset.

In [ ]:
%cd /content/
!wget "http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_snapshot_serengeti_2020_06_10.tar.gz"
!tar -xvf "faster_rcnn_resnet101_snapshot_serengeti_2020_06_10.tar.gz"

In [ ]:
#Modify path as required
pipeline_fname = "/content/faster_rcnn_resnet101_snapshot_serengeti_2020_06_10/pipeline.config"
fine_tune_checkpoint = "/content/faster_rcnn_resnet101_snapshot_serengeti_2020_06_10/model.ckpt"

In [ ]:
#Define training parameters

num_steps = 4000
batch_size = 4
num_eval_steps = 1000

## Configuring a Training Pipeline

In [ ]:
#Define variable pipeline_file here, if you want to use a custom configuration from https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [ ]:
%cd /content/models/research/

#### Attention - At this point make changes to the config file to work with your dataset (number of classes, learning rates, image aspect ratios etc).

#### The cells provided below work for the most part but are unreliable sometimes and need modification on a case by case basis.

#### Add these lines in the config file before training, they play an important role in creating context and embeddings required for context rcnn
from_detection_checkpoint: true
load_all_detection_checkpoint_vars: true
fine_tune_checkpoint_type: "detection"


Take a look at lines [108-131](https://github.com/prakrutpatel/Context-RCNN-Tortoises/blob/main/Examples/Faster%20RCNN.config) for reference

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [ ]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:

    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [ ]:
!cat {pipeline_fname}

In [ ]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

## Train the model

For some reason installing lvis may cause problems with tensorflow. If this happens rerun all the pip install lines at the start of this notebook (without restarting session).

Do not rerun this cell after doing that

In [ ]:
!pip install lvis

In [ ]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

In [ ]:
!ls {model_dir}

## Review training logs and tensorboard for metric and evaluation, redo training by changing configuration as needed.

In [ ]:
%tensorboard --logdir logs

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

#### We need to export this model twice.
1. With input_type image_tensor - Used when we pass in an image converted into 4-D tensor
2. With input_type tf_example - Used when we pass a 1-D string tensor containing serialized TFExample protos, used to create embeddings and context.


Read more at [Export Inference Graph](https://github.com/tensorflow/models/blob/master/research/object_detection/export_inference_graph.py)

In [ ]:
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path {pipeline_fname} \
    --output_directory "/content/Faster_RCNN_SS_imagetensor" \
    --trained_checkpoint_prefix {last_model_path} \
    --additional_output_tensor_names detection_features

In [ ]:
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type tf_example \
    --pipeline_config_path {pipeline_fname} \
    --output_directory "/content/Faster_RCNN_SS_tfexample" \
    --trained_checkpoint_prefix {last_model_path} \
    --additional_output_tensor_names detection_features

Exported model is located at '/content/{output directory name}'
Save as zip for future use

In [ ]:
!zip -r "/content/Faster_RCNN_SS_imagetensor.zip" "/content/Faster_RCNN_SS_imagetensor"
!zip -r "/content/Faster_RCNN_SS_tfexample.zip" "/content/Faster_RCNN_SS_tfexample"

#### Recommended - Save a copy of the zip file on the cloud and locally